# Data Load


In [1]:
import numpy as np
import pandas as pd
import math

from scipy import stats #Analysis 
from scipy.stats import norm 

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rcParams
%matplotlib inline

In [2]:
# 한글 폰트가 깨지면 실행하세요! 
import matplotlib.font_manager as fm
fontlist = fm.findSystemFonts(fontpaths = None, fontext='ttf')
# 아래 주석을 지워서 폰트 리스트를 확인하고 한글 폰트를 font_path에 추가합니다
fontlist[:]

from matplotlib import font_manager, rc
font_path = 'C:\\Users\\mtang\\AppData\\Local\\Microsoft\\Windows\\Fonts\\NanumSquare.ttf'
#font_path = 'C:\\WINDOWS\\Fonts\\NanumGothicLight.ttf'
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)

In [3]:
# pandas option 설정 하는 코드
# monthly_gain의 경우 부동소수점으로 나타나서 보기 어려울땐 윗 줄의 주석을 제거하고 아래에 주석을 추가하고
# 다시 원래대로 돌리고 싶다면 아래에 주석제거, 위 코드에 주석추가
pd.options.display.float_format = '{:.2f}'.format
#pd.reset_option('display.float_format')

In [4]:
path = "./Dataset/"

data = pd.read_csv(path+"kwproja_data_location.csv")

data

,DATE_,SHOP_CODE,SHOP_NAME,SHOP_TYPE_BIG,SHOP_TYPE_SMALL,LONGITUDE,LATITUDE,MONTHLY_GAIN,AVERAGE_SALE_PRICE,GU,DONG
0,201709,68281950112,대원갈비,뷔페,고기 뷔페,126.93,37.49,88292576.82,43319,관악구,신림동
1,201709,491680110,옛골토성,뷔페,고기 뷔페,126.95,37.66,25218858.16,63645,은평구,진관동
2,201708,4540650109,정든생삼겹살,뷔페,고기 뷔페,126.82,37.57,25957122.44,61605,강서구,방화1동
3,201708,42631710113,손가네생오리부페,뷔페,고기 뷔페,126.86,37.49,17709655.91,36938,구로구,개봉2동
4,201709,70824420108,한방복돼지,뷔페,고기 뷔페,127.13,37.53,24859801.35,58224,강동구,성내3동
...,...,...,...,...,...,...,...,...,...,...,...
2927734,201804,1709900344,신지도수산,수산물,회/수산물 일반,126.94,37.51,12570901.91,73033,동작구,노량진2동
2927735,201803,51063560503,노랑태 황태찜,수산물,회/수산물 일반,126.93,37.62,10489984.69,24402,은평구,불광2동
2927736,201803,5919500296,스마일수산,수산물,회/수산물 일반,126.92,37.48,31807992.77,16458,관악구,신사동
2927737,201805,4459300560,샛강,수산물,회/수산물 일반,126.94,37.51,7725388.62,71400,동작구,노량진2동


# 2. EDA 

2,927,739 rows X 11 columns

- 매장 속성 정보
  - shop_code -> 식별자 feature => drop   
  - shop_name => DL
  - longtitude : 경도, latitude : 위도 -> 매장 위치 (회사 근처, 학교 근처 등 매출 영향성 있음) -> K-mean clustering => geo, ML, DL
  - address1, address2 : GeoEncoder를 통해 따로 얻은 행정동, 1(30), 2(436) => DL
  - shop_type_big -> 15 category, shop_type_big_label, ML => DL
  - shop_type_small -> 61 category, shop_type_small_label, ML => DL 

- 매출 정보
  - date -> 24 category, 201606~ 201805 까지의 data
  - monthly_gain / avearge_sale_price = 한달 총 판매수

- 매출 통계 정보-> X

In [5]:
# rename data columns and check the data
data.columns = ['date', 'shop_code', 'shop_name', 'shop_type_big', 'shop_type_small', 
                'longitude', 'latitude', 'monthly_gain', 'average_sale_price', 'address1', 'address2']

print(data.columns, '\n')
print(data.info())

Index(['date', 'shop_code', 'shop_name', 'shop_type_big', 'shop_type_small',
       'longitude', 'latitude', 'monthly_gain', 'average_sale_price',
       'address1', 'address2'],
      dtype='object') 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2927739 entries, 0 to 2927738
Data columns (total 11 columns):
 #   Column              Dtype  
---  ------              -----  
 0   date                int64  
 1   shop_code           int64  
 2   shop_name           object 
 3   shop_type_big       object 
 4   shop_type_small     object 
 5   longitude           float64
 6   latitude            float64
 7   monthly_gain        float64
 8   average_sale_price  int64  
 9   address1            object 
 10  address2            object 
dtypes: float64(3), int64(3), object(5)
memory usage: 245.7+ MB
None


# 3. Preprocessing 
- df_geo : 구(address1)를 기준으로 통계량을 붙여놓은 데이터프레임 
    - monthly_gain + average_sale_price + shop_type_big 개수 
    - count : 데이터 개수 
    - mg : monthly_gain 약자 
    - sp : average_sale_price 약자 
    - mean, st, ~ max : 평균값, 표준편차, ~ 최대값 
    - shop_type_big(15) : 각 업종대분류에 해당하는 데이터가 해당 지역(구)에는 몇 개가 있는가? 
    - 0, 1, 2 : 각 구마다 1등, 2등, 3등으로 많은 업종 대분류는? 
    - **27 rows × 34 columns**
- df_shop : 업종 대분류(shop_type_big)을 기준으로 통계량을 붙여놓은 데이터프레임 
    - **15 rows × 49 columns**
- 구 -> label encoder => (30개)
    - '강남구' '강동구' '강북구' '강서구' '고양시 덕양구' '과천시' '관악구' '광명시' '광진구' '구로구' '구리시'
      '금천구' '노원구' '도봉구' '동대문구' '동작구' '마포구' '서대문구' '서초구' '성동구' '성북구' '송파구' '양천구'
      '영등포구' '용산구' '은평구' '종로구' '중구' '중랑구' '하남시' 

In [6]:
# 원본 data와 따로 관리 -> original data = data, preprocessed data = processed_data 
processed_data = data.copy()
processed_data

,date,shop_code,shop_name,shop_type_big,shop_type_small,longitude,latitude,monthly_gain,average_sale_price,address1,address2
0,201709,68281950112,대원갈비,뷔페,고기 뷔페,126.93,37.49,88292576.82,43319,관악구,신림동
1,201709,491680110,옛골토성,뷔페,고기 뷔페,126.95,37.66,25218858.16,63645,은평구,진관동
2,201708,4540650109,정든생삼겹살,뷔페,고기 뷔페,126.82,37.57,25957122.44,61605,강서구,방화1동
3,201708,42631710113,손가네생오리부페,뷔페,고기 뷔페,126.86,37.49,17709655.91,36938,구로구,개봉2동
4,201709,70824420108,한방복돼지,뷔페,고기 뷔페,127.13,37.53,24859801.35,58224,강동구,성내3동
...,...,...,...,...,...,...,...,...,...,...,...
2927734,201804,1709900344,신지도수산,수산물,회/수산물 일반,126.94,37.51,12570901.91,73033,동작구,노량진2동
2927735,201803,51063560503,노랑태 황태찜,수산물,회/수산물 일반,126.93,37.62,10489984.69,24402,은평구,불광2동
2927736,201803,5919500296,스마일수산,수산물,회/수산물 일반,126.92,37.48,31807992.77,16458,관악구,신사동
2927737,201805,4459300560,샛강,수산물,회/수산물 일반,126.94,37.51,7725388.62,71400,동작구,노량진2동


### geo

In [28]:
# monthly_gain 
group1 = processed_data[['monthly_gain', 'address1']].groupby('address1')
df1 = group1.describe().droplevel(axis=1, level=0)

# round
df1.iloc[:, 1:] = df1.iloc[:, 1:].applymap(lambda x : round(x, -4))
df1.columns = ['count', 'mean_mg', 'st_mg', 'min_mg', '25%_mg', '50%_mg', '75%_mg', 'max_mg']

# sum - 총 매출
df1['monthly_gain'] = group1.sum().apply(lambda x : round(x, -4))

df1.head()

,count,mean_mg,st_mg,min_mg,25%_mg,50%_mg,75%_mg,max_mg,monthly_gain
address1,,,,,,,,,
강남구,256562.00,52570000.00,808920000.00,0.00,9140000.00,21480000.00,44510000.00,91386200000.00,13486576890000.00
강동구,112134.00,23380000.00,172690000.00,0.00,3330000.00,8670000.00,20740000.00,10642020000.00,2621232850000.00
강북구,84071.00,18260000.00,122340000.00,0.00,2530000.00,6770000.00,17730000.00,9668020000.00,1534726850000.00
강서구,142272.00,34730000.00,357880000.00,0.00,4600000.00,12250000.00,29090000.00,23156200000.00,4941651280000.00
과천시,61.00,13960000.00,7970000.00,1340000.00,6190000.00,13440000.00,21100000.00,32190000.00,851370000.00


In [29]:
# average_sale_price
df2 = processed_data[['average_sale_price', 'address1']].groupby('address1')
df2 = df2.describe().droplevel(axis=1, level=0)

# round
df2.iloc[:, 1:] = df2.iloc[:, 1:].applymap(lambda x : round(x, -4))
df2.columns = ['count', 'mean_sp', 'st_sp', 'min_sp', '25%_sp', '50%_sp', '75%_sp', 'max_sp']
df2.head()

,count,mean_sp,st_sp,min_sp,25%_sp,50%_sp,75%_sp,max_sp
address1,,,,,,,,
강남구,256562.00,60000.00,320000.00,0.00,10000.00,20000.00,50000.00,50000000.00
강동구,112134.00,30000.00,140000.00,0.00,10000.00,20000.00,40000.00,17000000.00
강북구,84071.00,30000.00,90000.00,0.00,10000.00,20000.00,40000.00,10180000.00
강서구,142272.00,40000.00,120000.00,0.00,10000.00,20000.00,40000.00,7040000.00
과천시,61.00,30000.00,10000.00,10000.00,10000.00,20000.00,40000.00,50000.00


In [30]:
# shop_type_big
df3 = processed_data.groupby(['address1', 'shop_type_big']).size()
df3 = df3.unstack() # unstack : SQL 집계 결과를 가로, 세로 축으로 보기 좋게 나열 
df3 = df3.fillna(0) # NaN to 0
df3.head()

shop_type_big,다방/커피숍/카페,별식/퓨전요리,분식,뷔페,생활서비스,소매,수산물,양식,유흥주점,음식-미분류,일식,제과제빵떡케익,중식,패스트푸드,한식
address1,,,,,,,,,,,,,,,
강남구,32463.00,15488.00,14623.00,1524.00,0.00,29177.00,10846.00,1862.00,18621.00,47551.00,9873.00,7745.00,11426.00,11701.00,43662.00
강동구,10495.00,7458.00,8668.00,1152.00,0.00,15625.00,5414.00,285.00,10000.00,16628.00,2759.00,4141.00,3558.00,4809.00,21142.00
강북구,6917.00,6879.00,5571.00,861.00,0.00,10493.00,4422.00,130.00,8149.00,12582.00,1693.00,2489.00,2391.00,3331.00,18163.00
강서구,13003.00,10642.00,9763.00,1068.00,0.00,19298.00,7605.00,440.00,12928.00,21673.00,3356.00,5665.00,4910.00,6603.00,25318.00
과천시,0.00,24.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,13.00,24.00


In [31]:
# rank for shop_type_big
df4 = df3.apply(lambda s, n: pd.Series(s.nlargest(n).index), axis=1, n=3)
df4.head()

,0,1,2
address1,,,
강남구,음식-미분류,한식,다방/커피숍/카페
강동구,한식,음식-미분류,소매
강북구,한식,음식-미분류,소매
강서구,한식,음식-미분류,소매
과천시,별식/퓨전요리,한식,패스트푸드


In [32]:
df_geo = pd.concat([df1, df2.iloc[:, 1:]], axis=1)
df_geo = pd.concat([df_geo, df3], axis=1)
df_geo = pd.concat([df_geo, df4], axis=1)
df_geo.head()

,count,mean_mg,st_mg,min_mg,25%_mg,50%_mg,75%_mg,max_mg,monthly_gain,mean_sp,...,유흥주점,음식-미분류,일식,제과제빵떡케익,중식,패스트푸드,한식,0,1,2
address1,,,,,,,,,,,,,,,,,,,,,
강남구,256562.00,52570000.00,808920000.00,0.00,9140000.00,21480000.00,44510000.00,91386200000.00,13486576890000.00,60000.00,...,18621.00,47551.00,9873.00,7745.00,11426.00,11701.00,43662.00,음식-미분류,한식,다방/커피숍/카페
강동구,112134.00,23380000.00,172690000.00,0.00,3330000.00,8670000.00,20740000.00,10642020000.00,2621232850000.00,30000.00,...,10000.00,16628.00,2759.00,4141.00,3558.00,4809.00,21142.00,한식,음식-미분류,소매
강북구,84071.00,18260000.00,122340000.00,0.00,2530000.00,6770000.00,17730000.00,9668020000.00,1534726850000.00,30000.00,...,8149.00,12582.00,1693.00,2489.00,2391.00,3331.00,18163.00,한식,음식-미분류,소매
강서구,142272.00,34730000.00,357880000.00,0.00,4600000.00,12250000.00,29090000.00,23156200000.00,4941651280000.00,40000.00,...,12928.00,21673.00,3356.00,5665.00,4910.00,6603.00,25318.00,한식,음식-미분류,소매
과천시,61.00,13960000.00,7970000.00,1340000.00,6190000.00,13440000.00,21100000.00,32190000.00,851370000.00,30000.00,...,0.00,0.00,0.00,0.00,0.00,13.00,24.00,별식/퓨전요리,한식,패스트푸드


In [94]:
df_geo.to_csv('df_geo.csv', float_format = '%.2f', encoding = 'utf-8-sig')

### geo - date

In [33]:
# date 
df5 = processed_data[['date', 'address1', 'monthly_gain']].groupby(['address1', 'date']).mean().round(-4)
df6 = processed_data[['date', 'address1', 'monthly_gain']].groupby(['address1', 'date']).count()

df5 = df5.unstack()
df5 = df5.rename(columns = {'monthly_gain' : 'mean'})
df6 = df6.unstack()
df6 = df6.rename(columns = {'monthly_gain' : 'count'})

df_geo_date = pd.concat([df5, df6], axis=1)
df_geo_date.head()

mean                                                  \
date          201606      201607      201608      201609      201610   
address1                                                               
강남구      52340000.00 56110000.00 55070000.00 52060000.00 53380000.00   
강동구      23840000.00 24710000.00 24610000.00 24460000.00 23390000.00   
강북구      17820000.00 18520000.00 18560000.00 18790000.00 17970000.00   
강서구      35080000.00 36480000.00 35610000.00 35400000.00 35420000.00   
과천시      14010000.00 16770000.00 13570000.00 11760000.00 16940000.00   

                                                                      ...  \
date          201611      201612      201701      201702      201703  ...   
address1                                                              ...   
강남구      51310000.00 59010000.00 55350000.00 47080000.00 54360000.00  ...   
강동구      22560000.00 24850000.00 23760000.00 20950000.00 23870000.00  ...   
강북구      17460000.00 19870000.00 18550000.00 16770000.00 18770000.00  ...   
강서구      34340000.00 39170000.00 35120000.00 31190000.00 36630000.00  ...   
과천시      18090000.00 14190000.00 15780000.00 13670000.00 14210000.00  ...   

          count                                                                 
date     201708 201709 201710 201711 201712 201801 201802 201803 201804 201805  
address1                                                                        
강남구       10799  10764  10767  10778  10742  10616  10619  10512  10470  10548  
강동구        4707   4690   4693   4649   4688   4645   4634   4533   4550   4616  
강북구        3514   3490   3512   3498   3502   3478   3469   3438   3446   3497  
강서구        6014   5997   6017   6013   6009   5944   5917   5832   5841   5892  
과천시           2      2      2      2      2      2      2      2      2      2  

[5 rows x 48 columns]

In [34]:
df_geo_date.to_csv('df_geo_date.csv', float_format = '%.2f', encoding = 'utf-8-sig')

### shop type big

In [35]:
# monthly_gain 
group1 = processed_data[['monthly_gain', 'shop_type_big']].groupby('shop_type_big')
df1 = group1.describe().droplevel(axis=1, level=0)

df1.iloc[:, 1:] = df1.iloc[:, 1:].applymap(lambda x : round(x, -4))
df1.columns = ['count', 'mean_mg', 'st_mg', 'min_mg', '25%_mg', '50%_mg', '75%_mg', 'max_mg']

# sum - 총 매출
df1['monthly_gain'] = group1.sum().apply(lambda x : round(x, -4))

df1.head()

,count,mean_mg,st_mg,min_mg,25%_mg,50%_mg,75%_mg,max_mg,monthly_gain
shop_type_big,,,,,,,,,
다방/커피숍/카페,318339.00,13630000.00,26300000.00,0.00,3250000.00,7510000.00,15910000.00,2454380000.00,4339878830000.00
별식/퓨전요리,213619.00,19720000.00,28710000.00,0.00,5770000.00,12060000.00,23830000.00,1535740000.00,4212465640000.00
분식,188263.00,15050000.00,19540000.00,0.00,3470000.00,8710000.00,19690000.00,594570000.00,2834027670000.00
뷔페,24891.00,36000000.00,70220000.00,0.00,6550000.00,15850000.00,36200000.00,1246630000.00,896103630000.00
생활서비스,47.00,4970000.00,3440000.00,30000.00,2920000.00,4190000.00,6760000.00,15390000.00,233660000.00


In [36]:
# average_sale_price
df2 = processed_data[['average_sale_price', 'shop_type_big']].groupby('shop_type_big')
df2 = df2.describe().droplevel(axis=1, level=0)

df2.iloc[:, 1:] = df2.iloc[:, 1:].applymap(lambda x : round(x, -4))
df2.columns = ['count', 'mean_sp', 'st_sp', 'min_sp', '25%_sp', '50%_sp', '75%_sp', 'max_sp']
df2.head()

,count,mean_sp,st_sp,min_sp,25%_sp,50%_sp,75%_sp,max_sp
shop_type_big,,,,,,,,
다방/커피숍/카페,318339.00,10000.00,80000.00,0.00,10000.00,10000.00,10000.00,17520000.00
별식/퓨전요리,213619.00,30000.00,120000.00,0.00,20000.00,20000.00,30000.00,14280000.00
분식,188263.00,20000.00,60000.00,0.00,10000.00,10000.00,20000.00,15000000.00
뷔페,24891.00,70000.00,210000.00,0.00,20000.00,40000.00,60000.00,7450000.00
생활서비스,47.00,40000.00,40000.00,10000.00,20000.00,20000.00,60000.00,170000.00


In [37]:
# address1 - geo
df3 = processed_data.groupby(['shop_type_big', 'address1']).size()
df3 = df3.unstack() # unstack : SQL 집계 결과를 가로, 세로 축으로 보기 좋게 나열 
df3 = df3.fillna(0) # NaN to 0
df3.head()

address1,강남구,강동구,강북구,강서구,과천시,관악구,광명시,광진구,구로구,금천구,...,성동구,성북구,송파구,양천구,영등포구,용산구,은평구,종로구,중구,중랑구
shop_type_big,,,,,,,,,,,,,,,,,,,,,
다방/커피숍/카페,32463.00,10495.00,6917.00,13003.00,0.00,11065.00,0.00,10241.00,9044.00,7242.00,...,9320.00,11244.00,19248.00,8896.00,16601.00,12106.00,8714.00,18048.00,14653.00,6768.00
별식/퓨전요리,15488.00,7458.00,6879.00,10642.00,24.00,11099.00,0.00,7125.00,8372.00,5261.00,...,6175.00,8063.00,12996.00,8336.00,10300.00,5814.00,8473.00,6776.00,7214.00,7850.00
분식,14623.00,8668.00,5571.00,9763.00,0.00,7644.00,0.00,6169.00,7725.00,5136.00,...,4689.00,6543.00,12187.00,6991.00,9566.00,5359.00,7133.00,8032.00,7433.00,6096.00
뷔페,1524.00,1152.00,861.00,1068.00,0.00,1604.00,0.00,685.00,1238.00,978.00,...,759.00,855.00,1647.00,766.00,1211.00,629.00,1036.00,939.00,1005.00,1194.00
생활서비스,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,24.00,0.00,...,23.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [38]:
# rank for address1(geo)
df4 = df3.apply(lambda s, n: pd.Series(s.nlargest(n).index), axis=1, n=3)
df4.head()

,0,1,2
shop_type_big,,,
다방/커피숍/카페,강남구,마포구,서초구
별식/퓨전요리,강남구,송파구,마포구
분식,강남구,송파구,마포구
뷔페,송파구,관악구,강남구
생활서비스,구로구,성동구,강남구


In [39]:
df_type = pd.concat([df1, df2.iloc[:, 1:]], axis=1)
df_type = pd.concat([df_type, df3], axis=1)
df_type = pd.concat([df_type, df4], axis=1)
df_type.head()

,count,mean_mg,st_mg,min_mg,25%_mg,50%_mg,75%_mg,max_mg,monthly_gain,mean_sp,...,양천구,영등포구,용산구,은평구,종로구,중구,중랑구,0,1,2
shop_type_big,,,,,,,,,,,,,,,,,,,,,
다방/커피숍/카페,318339.00,13630000.00,26300000.00,0.00,3250000.00,7510000.00,15910000.00,2454380000.00,4339878830000.00,10000.00,...,8896.00,16601.00,12106.00,8714.00,18048.00,14653.00,6768.00,강남구,마포구,서초구
별식/퓨전요리,213619.00,19720000.00,28710000.00,0.00,5770000.00,12060000.00,23830000.00,1535740000.00,4212465640000.00,30000.00,...,8336.00,10300.00,5814.00,8473.00,6776.00,7214.00,7850.00,강남구,송파구,마포구
분식,188263.00,15050000.00,19540000.00,0.00,3470000.00,8710000.00,19690000.00,594570000.00,2834027670000.00,20000.00,...,6991.00,9566.00,5359.00,7133.00,8032.00,7433.00,6096.00,강남구,송파구,마포구
뷔페,24891.00,36000000.00,70220000.00,0.00,6550000.00,15850000.00,36200000.00,1246630000.00,896103630000.00,70000.00,...,766.00,1211.00,629.00,1036.00,939.00,1005.00,1194.00,송파구,관악구,강남구
생활서비스,47.00,4970000.00,3440000.00,30000.00,2920000.00,4190000.00,6760000.00,15390000.00,233660000.00,40000.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,구로구,성동구,강남구


In [18]:
df_type

,count,mean_mg,st_mg,min_mg,25%_mg,50%_mg,75%_mg,max_mg,monthly_gain,mean_sp,...,양천구,영등포구,용산구,은평구,종로구,중구,중랑구,0,1,2
shop_type_big,,,,,,,,,,,,,,,,,,,,,
다방/커피숍/카페,318339.00,13630000.00,26300000.00,0.00,3250000.00,7510000.00,15910000.00,2454380000.00,4339878830000.00,10000.00,...,8896.00,16601.00,12106.00,8714.00,18048.00,14653.00,6768.00,강남구,마포구,서초구
별식/퓨전요리,213619.00,19720000.00,28710000.00,0.00,5770000.00,12060000.00,23830000.00,1535740000.00,4212465640000.00,30000.00,...,8336.00,10300.00,5814.00,8473.00,6776.00,7214.00,7850.00,강남구,송파구,마포구
분식,188263.00,15050000.00,19540000.00,0.00,3470000.00,8710000.00,19690000.00,594570000.00,2834027670000.00,20000.00,...,6991.00,9566.00,5359.00,7133.00,8032.00,7433.00,6096.00,강남구,송파구,마포구
뷔페,24891.00,36000000.00,70220000.00,0.00,6550000.00,15850000.00,36200000.00,1246630000.00,896103630000.00,70000.00,...,766.00,1211.00,629.00,1036.00,939.00,1005.00,1194.00,송파구,관악구,강남구
생활서비스,47.00,4970000.00,3440000.00,30000.00,2920000.00,4190000.00,6760000.00,15390000.00,233660000.00,40000.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,구로구,성동구,강남구
소매,359034.00,116400000.00,1569080000.00,0.00,7380000.00,27180000.00,47730000.00,181237340000.00,41790631240000.00,40000.00,...,13464.00,16985.00,9487.00,13462.00,7117.00,14789.00,12161.00,강남구,송파구,강서구
수산물,131323.00,28340000.00,33010000.00,0.00,8460000.00,18100000.00,36320000.00,703400000.00,3721432720000.00,60000.00,...,3756.00,8149.00,3330.00,4618.00,4958.00,5433.00,5283.00,강남구,영등포구,송파구
양식,13726.00,25500000.00,38190000.00,10000.00,6090000.00,13860000.00,28730000.00,550880000.00,349946020000.00,50000.00,...,432.00,655.00,807.00,308.00,989.00,459.00,293.00,강남구,마포구,서초구
유흥주점,254766.00,15800000.00,22750000.00,0.00,3690000.00,9030000.00,19230000.00,1289820000.00,4026153940000.00,50000.00,...,7714.00,12527.00,9559.00,8768.00,9901.00,8426.00,9978.00,마포구,강남구,송파구


### Total

In [19]:
df_total = processed_data[['monthly_gain', 'average_sale_price']].describe().transpose()

# round - except for min, count 
df_total.iloc[:, 1:3] = df_total.iloc[:, 1:3].applymap(lambda x : round(x,  -4))
df_total.iloc[:, 4:] = df_total.iloc[:, 4:].applymap(lambda x : round(x,  -4))

df_total

,count,mean,std,min,25%,50%,75%,max
monthly_gain,2927739.00,32770000.00,552030000.00,5.06,4310000.00,11580000.00,27580000.00,181237340000.00
average_sale_price,2927739.00,40000.00,180000.00,1.00,10000.00,20000.00,40000.00,165260000.00


In [20]:
sum_mg = processed_data['monthly_gain'].sum().round(-4)
sum_sp = processed_data['average_sale_price'].sum().round(-4)

df_total["sum"] = [sum_mg, sum_sp]

df_total

,count,mean,std,min,25%,50%,75%,max,sum
monthly_gain,2927739.00,32770000.00,552030000.00,5.06,4310000.00,11580000.00,27580000.00,181237340000.00,95953120370000.00
average_sale_price,2927739.00,40000.00,180000.00,1.00,10000.00,20000.00,40000.00,165260000.00,111459210000.00


In [21]:
df_total.to_csv('df_total.csv', float_format = '%.2f', encoding = 'utf-8-sig')

### shop_name + shop_type_big + shop_type_small : df_nlp

In [7]:
# NLP용 preprocessing 
# shop_name, shop_type_big, shop_type_small = concat_text 
df_nlp1 = processed_data.copy()
df_nlp1['concat_text'] = df_nlp1['shop_name'] + ' ' + df_nlp1['shop_type_big'] + ' ' + df_nlp1['shop_type_small'] + ' '
df_nlp1.head()

,date,shop_code,shop_name,shop_type_big,shop_type_small,longitude,latitude,monthly_gain,average_sale_price,address1,address2,concat_text
0,201709,68281950112,대원갈비,뷔페,고기 뷔페,126.93,37.49,88292576.82,43319,관악구,신림동,대원갈비 뷔페 고기 뷔페
1,201709,491680110,옛골토성,뷔페,고기 뷔페,126.95,37.66,25218858.16,63645,은평구,진관동,옛골토성 뷔페 고기 뷔페
2,201708,4540650109,정든생삼겹살,뷔페,고기 뷔페,126.82,37.57,25957122.44,61605,강서구,방화1동,정든생삼겹살 뷔페 고기 뷔페
3,201708,42631710113,손가네생오리부페,뷔페,고기 뷔페,126.86,37.49,17709655.91,36938,구로구,개봉2동,손가네생오리부페 뷔페 고기 뷔페
4,201709,70824420108,한방복돼지,뷔페,고기 뷔페,127.13,37.53,24859801.35,58224,강동구,성내3동,한방복돼지 뷔페 고기 뷔페


In [8]:
df_nlp2 = df_nlp1[['concat_text', 'address1']]
df_nlp2.head()

,concat_text,address1
0,대원갈비 뷔페 고기 뷔페,관악구
1,옛골토성 뷔페 고기 뷔페,은평구
2,정든생삼겹살 뷔페 고기 뷔페,강서구
3,손가네생오리부페 뷔페 고기 뷔페,구로구
4,한방복돼지 뷔페 고기 뷔페,강동구


In [9]:
from keras.preprocessing.text import Tokenizer

df_nlp = pd.DataFrame()
alist = df_nlp2['address1'].unique()
for add in alist : 
    tk = Tokenizer()
    tk.fit_on_texts(df_nlp2[df_nlp2['address1'] == add]['concat_text'])
    
    wordlist = sorted(tk.word_counts.items(), key=lambda x: x[1], reverse=True)[:30]
    df_word = pd.DataFrame(wordlist)
    df_nlp = pd.concat([df_nlp, df_word], axis=1)
    
    print(add, ": ", list(wordlist))
    print("\nvocab words 개수 : ", len(tk.word_index.items()))

관악구 :  [('전문점', 57749), ('미분류', 36817), ('음식', 36642), ('한식', 22065), ('카페', 20593), ('소매', 14644), ('분식', 12041), ('별식', 11988), ('퓨전요리', 11988), ('유흥주점', 11914), ('다방', 10863), ('커피숍', 10863), ('디저트', 9524), ('수산물', 9517), ('편의점', 7402), ('치킨', 7160), ('닭강정', 7040), ('일반', 6999), ('패스트푸드', 6559), ('고기구이', 6372), ('육회', 6348), ('식료품', 5038), ('중식', 5000), ('호프', 4992), ('맥주', 4840), ('백반', 4557), ('한정식', 4536), ('오리', 4192), ('닭요리', 4168), ('포장마차', 3868)]

vocab words 개수 :  6382
은평구 :  [('전문점', 44039), ('미분류', 30312), ('음식', 30204), ('한식', 17738), ('카페', 17604), ('소매', 13637), ('분식', 11559), ('유흥주점', 9451), ('커피숍', 8863), ('다방', 8794), ('디저트', 8294), ('수산물', 7773), ('별식', 7120), ('퓨전요리', 7120), ('편의점', 6170), ('일반', 5664), ('고기구이', 5006), ('육회', 5006), ('식료품', 4512), ('닭강정', 4239), ('치킨', 4218), ('호프', 4083), ('패스트푸드', 4062), ('맥주', 3981), ('백반', 3641), ('한정식', 3617), ('오리', 3485), ('닭요리', 3485), ('제과제빵떡케익', 3391), ('중식', 3335)]

vocab words 개수 :  5401
강서구 :  [('전문점', 65346), ('미분류', 

서대문구 :  [('전문점', 43068), ('미분류', 30559), ('음식', 30480), ('카페', 21680), ('한식', 15417), ('다방', 10929), ('커피숍', 10891), ('소매', 10422), ('디저트', 10170), ('별식', 9654), ('퓨전요리', 9654), ('유흥주점', 9104), ('분식', 8303), ('치킨', 6162), ('닭강정', 6100), ('편의점', 5310), ('패스트푸드', 5173), ('수산물', 4895), ('고기구이', 4348), ('육회', 4348), ('일반', 4234), ('호프', 3839), ('한정식', 3821), ('백반', 3821), ('맥주', 3742), ('식료품', 3394), ('중식', 3232), ('주', 3230), ('오리', 3092), ('닭요리', 3092)]

vocab words 개수 :  5289
양천구 :  [('전문점', 45026), ('미분류', 24688), ('음식', 24548), ('카페', 16790), ('한식', 14275), ('소매', 13029), ('분식', 12406), ('다방', 8630), ('커피숍', 8606), ('별식', 8570), ('퓨전요리', 8570), ('디저트', 8096), ('유흥주점', 7350), ('수산물', 6449), ('패스트푸드', 6056), ('제과제빵떡케익', 5919), ('편의점', 5608), ('일반', 5516), ('식료품', 4733), ('닭강정', 4643), ('제과', 4628), ('제빵', 4628), ('치킨', 4595), ('고기구이', 3947), ('육회', 3947), ('닭요리', 3879), ('오리', 3859), ('호프', 3527), ('맥주', 3425), ('중식', 3110)]

vocab words 개수 :  4731
노원구 :  [('전문점', 46216), ('미분류', 27209)

In [55]:
cols_list = list(df_nlp.columns)
cnt = 0
for i in range (len(cols_list)) :
    if i%2 == 0 : 
        cols_list[i] = alist[cnt]
        cnt = cnt + 1 
    else : 
        cols_list[i] = str(alist[cnt-1]) + " counts"
df_nlp.columns = cols_list
df_nlp.columns

Index(['관악구', '관악구 counts', '은평구', '은평구 counts', '강서구', '강서구 counts', '구로구',
       '구로구 counts', '강동구', '강동구 counts', '중구', '중구 counts', '종로구',
       '종로구 counts', '송파구', '송파구 counts', '서초구', '서초구 counts', '성북구',
       '성북구 counts', '용산구', '용산구 counts', '금천구', '금천구 counts', '강북구',
       '강북구 counts', '동대문구', '동대문구 counts', '광진구', '광진구 counts', '성동구',
       '성동구 counts', '마포구', '마포구 counts', '서대문구', '서대문구 counts', '양천구',
       '양천구 counts', '노원구', '노원구 counts', '중랑구', '중랑구 counts', '영등포구',
       '영등포구 counts', '동작구', '동작구 counts', '강남구', '강남구 counts', '도봉구',
       '도봉구 counts', '과천시', '과천시 counts', '광명시', '광명시 counts'],
      dtype='object')

In [56]:
df_nlp

,관악구,관악구 counts,은평구,은평구 counts,강서구,강서구 counts,구로구,구로구 counts,강동구,강동구 counts,...,동작구,동작구 counts,강남구,강남구 counts,도봉구,도봉구 counts,과천시,과천시 counts,광명시,광명시 counts
0,전문점,57749,전문점,44039,전문점,65346,전문점,48644,전문점,48571,...,전문점,43203,전문점,106880,전문점,32800,전문점,37.00,음식,18.00
1,미분류,36817,미분류,30312,미분류,45495,미분류,34625,미분류,34346,...,미분류,24939,미분류,94828,미분류,19580,bbq,24.00,미분류,18.00
2,음식,36642,음식,30204,음식,45112,음식,34552,음식,34158,...,음식,24898,음식,94658,음식,19468,치킨앤비어,24.00,먹빵,9.00
3,한식,22065,한식,17738,한식,26397,한식,19828,카페,20257,...,카페,19255,카페,63385,한식,13763,별식,24.00,NaN,NaN
4,카페,20593,카페,17604,카페,24955,카페,18281,한식,20197,...,한식,15871,한식,42806,카페,11084,퓨전요리,24.00,NaN,NaN
5,소매,14644,소매,13637,소매,19512,소매,14255,소매,15317,...,소매,11428,다방,31737,소매,9899,치킨,24.00,NaN,NaN
6,분식,12041,분식,11559,분식,15237,분식,11939,분식,15118,...,분식,10519,커피숍,31737,분식,8946,닭강정,24.00,NaN,NaN
7,별식,11988,유흥주점,9451,유흥주점,12972,유흥주점,10135,커피숍,10280,...,다방,9918,디저트,30002,유흥주점,6194,정성한끼,13.00,NaN,NaN
8,퓨전요리,11988,커피숍,8863,다방,12411,다방,9152,다방,10262,...,커피숍,9905,소매,29621,커피숍,5776,패스트푸드,13.00,NaN,NaN
9,유흥주점,11914,다방,8794,커피숍,12411,커피숍,9130,디저트,9537,...,별식,9661,분식,22244,다방,5756,도시락,13.00,NaN,NaN


In [57]:
df_nlp.to_csv('df_nlp.csv',encoding = 'utf-8-sig')

### shop_name

In [58]:
# NLP용 preprocessing 
# shop_name, shop_type_big, shop_type_small = concat_text 
df_nlp1 = processed_data.copy()
df_nlp1.head()

,date,shop_code,shop_name,shop_type_big,shop_type_small,longitude,latitude,monthly_gain,average_sale_price,address1,address2
0,201709,68281950112,대원갈비,뷔페,고기 뷔페,126.93,37.49,88292576.82,43319,관악구,신림동
1,201709,491680110,옛골토성,뷔페,고기 뷔페,126.95,37.66,25218858.16,63645,은평구,진관동
2,201708,4540650109,정든생삼겹살,뷔페,고기 뷔페,126.82,37.57,25957122.44,61605,강서구,방화1동
3,201708,42631710113,손가네생오리부페,뷔페,고기 뷔페,126.86,37.49,17709655.91,36938,구로구,개봉2동
4,201709,70824420108,한방복돼지,뷔페,고기 뷔페,127.13,37.53,24859801.35,58224,강동구,성내3동


In [59]:
from keras.preprocessing.text import Tokenizer

df_nlp = pd.DataFrame()
alist = df_nlp1['address1'].unique()
for add in alist : 
    tk = Tokenizer()
    tk.fit_on_texts(df_nlp1[df_nlp1['address1'] == add]['shop_name'])
    
    wordlist = sorted(tk.word_counts.items(), key=lambda x: x[1], reverse=True)[:30]
    df_word = pd.DataFrame(wordlist)
    df_nlp = pd.concat([df_nlp, df_word], axis=1)
    
    print(add, ": ", list(wordlist))
    print("\nvocab words 개수 : ", len(tk.word_index.items()))

관악구 :  [('주', 2193), ('신림점', 2122), ('gs25', 1357), ('씨유', 1058), ('피자스쿨', 831), ('신림역점', 811), ('서울대입구역점', 789), ('봉천점', 777), ('세븐일레븐', 764), ('호치킨', 717), ('맛닭꼬', 665), ('주식회사', 650), ('낙성대점', 646), ('중국성', 626), ('난곡점', 558), ('코리아세븐', 547), ('서울대점', 513), ('59쌀피자', 513), ('호식이두마리치킨', 488), ('cafe', 460), ('cu', 457), ('파리바게뜨', 444), ('소문난순대국', 425), ('이마트24', 400), ('신대방역점', 390), ('관악점', 376), ('사당역점', 374), ('태능숯불갈비', 373), ('보라매점', 370), ('후라이드참잘하는집', 354)]

vocab words 개수 :  6311
은평구 :  [('gs25', 1687), ('주', 1651), ('연신내점', 1422), ('씨유', 987), ('불광점', 727), ('세븐일레븐', 727), ('응암점', 679), ('주식회사', 540), ('골목식당', 520), ('코리아세븐', 520), ('은평뉴타운점', 518), ('카페', 516), ('역촌점', 443), ('은평점', 433), ('구파발역점', 413), ('파리바게뜨', 363), ('충무김밥', 336), ('이디야', 332), ('구산역점', 323), ('cu', 321), ('응암역점', 313), ('갈현점', 311), ('하우돈곱창', 310), ('연신내역점', 309), ('착한정육점', 308), ('구산점', 297), ('연희김밥', 297), ('떡볶이', 296), ('진로마트', 288), ('커피', 288)]

vocab words 개수 :  5335
강서구 :  [('주', 3563), ('gs25', 2

서대문구 :  [('주', 3230), ('신촌점', 2169), ('페리카나치킨', 1285), ('주식회사', 1239), ('gs25', 1048), ('또래오래', 913), ('봉구비어', 907), ('깐부치킨', 711), ('이대점', 674), ('홍제점', 667), ('카페', 624), ('연희점', 593), ('명지대점', 588), ('씨유', 551), ('자매식당', 519), ('가재울점', 453), ('cu', 446), ('세븐일레븐', 440), ('커피', 409), ('코리아세븐', 393), ('소문난집', 390), ('coffee', 388), ('cafe', 376), ('파리바게뜨', 353), ('북가좌점', 333), ('홍제역점', 311), ('서대문역점', 286), ('서대문점', 269), ('이디야', 249), ('남가좌점', 231)]

vocab words 개수 :  5211
양천구 :  [('파리바게뜨', 2661), ('주', 2351), ('목동점', 2120), ('gs25', 1124), ('배스킨라빈스', 1108), ('피자마루', 876), ('비비큐', 708), ('신월점', 656), ('춘천닭갈비', 615), ('코리아세븐', 586), ('봉구스밥버거', 578), ('양천구청점', 569), ('신정점', 542), ('오목교점', 533), ('국수나무', 485), ('주식회사', 472), ('이마트24', 469), ('씨유', 463), ('세븐일레븐', 402), ('cu', 382), ('cafe', 378), ('목동역점', 366), ('명성왕족발', 338), ('자금성', 336), ('우리분식', 327), ('목동', 301), ('서울식당', 265), ('우정식당', 251), ('치킨사냥', 248), ('목동파리공원점', 244)]

vocab words 개수 :  4649
노원구 :  [('주', 1449), ('gs25', 133

In [60]:
cols_list = list(df_nlp.columns)
cnt = 0
for i in range (len(cols_list)) :
    if i%2 == 0 : 
        cols_list[i] = alist[cnt]
        cnt = cnt + 1 
    else : 
        cols_list[i] = str(alist[cnt-1]) + " counts"
df_nlp.columns = cols_list
df_nlp.columns

Index(['관악구', '관악구 counts', '은평구', '은평구 counts', '강서구', '강서구 counts', '구로구',
       '구로구 counts', '강동구', '강동구 counts', '중구', '중구 counts', '종로구',
       '종로구 counts', '송파구', '송파구 counts', '서초구', '서초구 counts', '성북구',
       '성북구 counts', '용산구', '용산구 counts', '금천구', '금천구 counts', '강북구',
       '강북구 counts', '동대문구', '동대문구 counts', '광진구', '광진구 counts', '성동구',
       '성동구 counts', '마포구', '마포구 counts', '서대문구', '서대문구 counts', '양천구',
       '양천구 counts', '노원구', '노원구 counts', '중랑구', '중랑구 counts', '영등포구',
       '영등포구 counts', '동작구', '동작구 counts', '강남구', '강남구 counts', '도봉구',
       '도봉구 counts', '과천시', '과천시 counts', '광명시', '광명시 counts'],
      dtype='object')

In [61]:
df_nlp

,관악구,관악구 counts,은평구,은평구 counts,강서구,강서구 counts,구로구,구로구 counts,강동구,강동구 counts,...,동작구,동작구 counts,강남구,강남구 counts,도봉구,도봉구 counts,과천시,과천시 counts,광명시,광명시 counts
0,주,2193,gs25,1687,주,3563,주,2105,주,2518,...,주,1928,주,17411,창동점,951,bbq,24.00,먹빵,9.00
1,신림점,2122,주,1651,gs25,2342,구로점,1266,천호점,1464,...,노량진점,1581,주식회사,7799,주,802,치킨앤비어,24.00,NaN,NaN
2,gs25,1357,연신내점,1422,마곡점,1690,gs25,1227,gs25,1405,...,gs25,1265,gs25,4230,가장맛있는족발,797,정성한끼,13.00,NaN,NaN
3,씨유,1058,씨유,987,씨유,1362,구로디지털점,901,할매순대국,1292,...,교촌치킨,1103,역삼점,2322,종로빈대떡,751,NaN,NaN,NaN,NaN
4,피자스쿨,831,불광점,727,화곡점,1288,신도림점,893,길동점,944,...,명동찌개마을,995,코리아세븐,2168,김가네김밥,702,NaN,NaN,NaN,NaN
5,신림역점,811,세븐일레븐,727,주식회사,1193,주식회사,696,주식회사,899,...,노랑통닭,973,대치점,1819,gs25,678,NaN,NaN,NaN,NaN
6,서울대입구역점,789,응암점,679,등촌점,914,고척점,685,명일점,721,...,상도점,895,카페,1679,쌍문점,665,NaN,NaN,NaN,NaN
7,봉천점,777,주식회사,540,발산역점,896,개봉점,681,씨유,672,...,중앙대점,861,강남점,1578,세븐일레븐,579,NaN,NaN,NaN,NaN
8,세븐일레븐,764,골목식당,520,세븐일레븐,877,씨유,665,암사점,664,...,주식회사,782,씨유,1512,주식회사,530,NaN,NaN,NaN,NaN
9,호치킨,717,코리아세븐,520,발산점,859,cu,604,코리아세븐,662,...,세븐일레븐,647,삼성점,1511,방학점,510,NaN,NaN,NaN,NaN


In [62]:
df_nlp.to_csv('df_shopname.csv',encoding = 'utf-8-sig')